In [2]:
# Imports and data load (use the provided housing CSV)
import numpy as np, pandas as pd, os, math, random
np.random.seed(42)
df = pd.read_csv('/content/housing (1).csv')
X_full = df.drop(columns=['MEDV']).to_numpy(dtype=float)
y_full = df['MEDV'].to_numpy(dtype=float).reshape(-1,1)
print("Loaded dataset:", X_full.shape, y_full.shape)


# Define NN class (NumPy-only)
class NN:
    def __init__(self, input_size, hidden_size, output_size=1, lr=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr
        limit1 = np.sqrt(6 / (input_size + hidden_size))
        limit2 = np.sqrt(6 / (hidden_size + output_size))
        self.W1 = np.random.uniform(-limit1, limit1, (input_size, hidden_size))
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.uniform(-limit2, limit2, (hidden_size, output_size))
        self.b2 = np.zeros((1, output_size))
    def sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))
    def sigmoid_deriv(self, s):
        return s * (1 - s)
    def forward(self, X):
        z1 = np.dot(X, self.W1) + self.b1
        a1 = self.sigmoid(z1)
        z2 = np.dot(a1, self.W2) + self.b2
        yhat = z2
        cache = {'X': X, 'z1': z1, 'a1': a1, 'z2': z2}
        return yhat, cache
    def compute_loss(self, yhat, y):
        return np.mean((yhat - y) ** 2)
    def backward(self, cache, yhat, y):
        n = y.shape[0]
        a1 = cache['a1']
        X = cache['X']
        dz2 = (2.0 * (yhat - y)) / n
        dW2 = np.dot(a1.T, dz2)
        db2 = np.sum(dz2, axis=0, keepdims=True)
        da1 = np.dot(dz2, self.W2.T)
        dz1 = da1 * self.sigmoid_deriv(a1)
        dW1 = np.dot(X.T, dz1)
        db1 = np.sum(dz1, axis=0, keepdims=True)
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
    def fit(self, X, y, epochs=1000, verbose=False):
        history = {'loss': []}
        for ep in range(epochs):
            yhat, cache = self.forward(X)
            loss = self.compute_loss(yhat, y)
            history['loss'].append(loss)
            self.backward(cache, yhat, y)
            if verbose and (ep % 100 == 0 or ep==epochs-1):
                print(f"Epoch {ep+1}/{epochs}  loss={loss:.6f}")
        return history
    def predict(self, X):
        yhat, _ = self.forward(X)
        return yhat


# K-fold split and scaling helpers (min-max using training fold)
def k_fold_split(X, y, k=5, seed=42):
    n = X.shape[0]
    idx = np.arange(n)
    rng = np.random.RandomState(seed)
    rng.shuffle(idx)
    folds = []
    fold_sizes = [(n // k) + (1 if i < (n % k) else 0) for i in range(k)]
    current = 0
    for fs in fold_sizes:
        test_idx = idx[current: current+fs]
        train_idx = np.setdiff1d(idx, test_idx)
        folds.append((train_idx, test_idx))
        current += fs
    return folds
def fit_scaler(X):
    min_ = X.min(axis=0, keepdims=True)
    max_ = X.max(axis=0, keepdims=True)
    range_ = np.where(max_ - min_ == 0, 1.0, max_ - min_)
    return {'min': min_, 'range': range_}
def transform_scaler(X, scaler):
    return (X - scaler['min']) / scaler['range']


# Run CV experiment function (this will be used below)
def run_cv_experiment(X_full, y_full, hidden_neurons, lr, k=5, epochs=1000, seed=42):
    folds = k_fold_split(X_full, y_full, k=k, seed=seed)
    val_losses = []
    for i, (train_idx, test_idx) in enumerate(folds):
        X_train, y_train = X_full[train_idx], y_full[train_idx]
        X_test, y_test = X_full[test_idx], y_full[test_idx]
        X_scaler = fit_scaler(X_train)
        y_scaler = fit_scaler(y_train)
        X_train_s = transform_scaler(X_train, X_scaler)
        X_test_s = transform_scaler(X_test, X_scaler)
        y_train_s = transform_scaler(y_train, y_scaler)
        y_test_s = transform_scaler(y_test, y_scaler)
        model = NN(input_size=X_train_s.shape[1], hidden_size=hidden_neurons, output_size=1, lr=lr)
        model.fit(X_train_s, y_train_s, epochs=epochs, verbose=False)
        ypred = model.predict(X_test_s)
        loss = model.compute_loss(ypred, y_test_s)
        val_losses.append(loss)
    val_losses = np.array(val_losses)
    return {'mean_mse': float(val_losses.mean()), 'std_mse': float(val_losses.std()), 'fold_losses': val_losses.tolist()}


# Run the three settings for 5-fold and 10-fold CV (1000 epochs)
settings = [
    {'hidden':3, 'lr':0.01},
    {'hidden':4, 'lr':0.001},
    {'hidden':5, 'lr':0.0001},
]
results = {}
for s in settings:
    for k in [5,10]:
        print(f"Running: hidden={s['hidden']}, lr={s['lr']}, k={k}")
        res = run_cv_experiment(X_full, y_full, hidden_neurons=s['hidden'], lr=s['lr'], k=k, epochs=1000)
        print(f" -> mean MSE (scaled target) = {res['mean_mse']:.6f}, std = {res['std_mse']:.6f}\\n")
        results[(s['hidden'], s['lr'], k)] = res


Loaded dataset: (489, 3) (489, 1)
Running: hidden=3, lr=0.01, k=5
 -> mean MSE (scaled target) = 0.024886, std = 0.011573\n
Running: hidden=3, lr=0.01, k=10
 -> mean MSE (scaled target) = 0.035471, std = 0.014229\n
Running: hidden=4, lr=0.001, k=5
 -> mean MSE (scaled target) = 0.036220, std = 0.018449\n
Running: hidden=4, lr=0.001, k=10
 -> mean MSE (scaled target) = 0.029660, std = 0.009780\n
Running: hidden=5, lr=0.0001, k=5
 -> mean MSE (scaled target) = 0.330732, std = 0.508923\n
Running: hidden=5, lr=0.0001, k=10
 -> mean MSE (scaled target) = 0.241056, std = 0.195116\n


In [ ]:
from google.colab import drive
drive.mount('/content/drive')